In [12]:
import torch

from transformers import pipeline

from llama_index import LLMPredictor
from llama_index import GPTListIndex, GPTSimpleVectorIndex, LangchainEmbedding, PromptHelper, SimpleDirectoryReader

from langchain.llms.base import LLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [6]:
class FlanLLM(LLM):
    model_name = 'google/flan-t5-base'
    pipeline = pipeline(
        'text2text-generation',
        model=model_name,
        # device=0,
        model_kwargs={'torch_dtype': torch.bfloat16}
    )

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]['generated_text']
    
    def _identifying_params(self):
        return {'name_of_model': self.model_name}
    
    def _llm_type(self):
        return 'custom'
    
llm_predictor = LLMPredictor(llm=FlanLLM())

In [7]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [8]:
text1 = '''
The next generation of AI for developers and Google Workspace
Mar 14, 2023

3 min read

We’re bringing the power of generative AI to developers and businesses through Google Cloud and MakerSuite. And we’re introducing new AI-powered features in Google Workspace.

Thomas Kurian
CEO, Google Cloud
Moving lines and dots in the 4 Google colors

Google has been investing in AI for many years and bringing its benefits to individuals, businesses and communities. Whether it’s publishing state-of-the-art research, building helpful products or developing tools and resources that enable others, we’re committed to making AI accessible to everyone.

We’re now at a pivotal moment in our AI journey. Breakthroughs in generative AI are fundamentally changing how people interact with technology — and at Google, we’ve been responsibly developing large language models so we can safely bring them to our products. Today, we’re excited to share our early progress. Developers and businesses can now try new APIs and products that make it easy, safe and scalable to start building with Google’s best AI models through Google Cloud and a new prototyping environment called MakerSuite. And in Google Workspace, we’re introducing new features that help people harness the power of generative AI to create, connect and collaborate.

PaLM API & MakerSuite: An approachable way to explore and prototype with generative AI applications

So many technology and platform shifts — from mobile to cloud computing — have inspired entire ecosystems of developers to start new businesses, imagine new products, and transform how they create. We’re in the midst of another shift with AI that is having a profound effect on every industry.

For developers who are experimenting with AI, we’re introducing the PaLM API, an easy and safe way to build on top of our best language models. Today, we’re making an efficient model available, in terms of size and capabilities, and we’ll add other sizes soon. The API also comes with an intuitive tool called MakerSuite, which lets you quickly prototype ideas and, over time, will have features for prompt engineering, synthetic data generation and custom-model tuning — all supported by robust safety tools. Select developers can access the PaLM API and MakerSuite in Private Preview today, and stay tuned for our waitlist soon.

Bringing generative AI capabilities to Google Cloud

For developers who want to build and customize their own models and apps using generative AI, they can access Google’s AI models, including PaLM, on Google Cloud. We’re bringing new generative AI capabilities to our Google Cloud AI portfolio to help developers and organizations access enterprise-level safety, security, privacy, as well as integrate with their existing Cloud solutions:

Generative AI support in Vertex AI: Developers and businesses already use Google Cloud’s Vertex AI platform to build and deploy machine learning models and AI applications at scale. We are now providing foundation models, initially for generating text and images, and over time with audio and video. Google Cloud customers will have the ability to discover models, create and modify prompts, fine tune them with their own data, and deploy applications that use these powerful new technologies.

Generative AI App Builder: Businesses and governments also want to build their own AI-powered chat interfaces and digital assistants. To enable this, we are introducing Generative AI App Builder, which connects conversational AI flows with out of the box search experiences and foundation models — helping companies build generative AI applications in minutes or hours.

New AI partnerships and programs: In addition to announcing new Google Cloud AI products, we’re also committed to being the most open cloud provider. We’re expanding our AI ecosystem and specialized programs for technology partners, AI-focused software providers and startups.

Starting today, trusted testers are accessing Vertex AI with Generative AI support and Generative AI App Builder.

New generative AI features in Workspace

More than 3 billion people already benefit from AI-powered features in Google Workspace, whether it’s using Smart Compose in Gmail or auto-generated summaries in Google Docs. Now, we’re excited to take the next step and bring a limited set of trusted testers a new set of features that makes the process of writing even easier. In Gmail and Google Docs, you can simply type in a topic you’d like to write about, and a draft will be instantly generated for you. So if you’re a manager onboarding a new employee, Workspace saves you the time and effort involved in writing that first welcome email. From there, you can elaborate upon or abbreviate the message or adjust the tone to be more playful or professional — all in just a few clicks. We’ll be rolling out these new experiences to testers in the coming weeks.

Generative AI in Docs helping to write a job description.

Scaling AI responsibly

Generative AI is an emerging and rapidly evolving technology with complex challenges. That’s why we invite external and internal testers to pressure test new experiences, and why we have AI Principles to guide this work. These principles also serve as an ongoing commitment to our customers worldwide who rely on our products to build and grow their businesses safely with AI. Our goal is to continue to be bold and responsible in our approach and partner with others to improve our AI models so they’re safe and helpful for everyone.

We’re so excited by the potential of generative AI, and the opportunities it will unlock — from helping people express themselves creatively, to helping developers build brand new types of applications, to transforming how businesses and governments engage their customers and constituents. Stay tuned for more to come in the weeks and months ahead.

POSTED IN:
AI Google Cloud Google Workspace

About the author
Thomas Kurian
CEO, Google Cloud

Thomas joined Google in November 2018 as the CEO of Google Cloud. Prior to Google, Thomas spent 22 years at Oracle, where most recently he was President of Product Development. Before that, Thomas worked at McKinsey as a business analyst and engagement manager. His nearly 30 years of experience have given him a deep knowledge of engineering, enterprise relationships, and leadership of large organizations.
'''

In [9]:
from llama_index import Document

# documents = SimpleDirectoryReader('data').load_data()

text_list = [text1]
documents = [Document(t) for t in text_list]

In [16]:
# # Set number of output tokens
# num_output = 250

# # Set maximun input size
# max_input_size = 512

# # Set maximun chunk overlap
# max_chunk_overlap = 20

# prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [17]:
# index = GPTSimpleVectorIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)
# index = GPTListIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor, prompt_helper=prompt_helper)
index = GPTListIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

# index.save_to_disk('index.json')

In [14]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [15]:
response = index.query('Who wrote this document?')
# response = index.query('What is this document about?')
# response = index.query('What is the most important point in this document?')
# response = index.query('Can you summarize this document?')
print(response)

Thomas Kurian ------------
